# The Event and Schedule classes

You should copy this code into your own Python notebook and use it to complete your pre-class work.

In [121]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import heapq

In [122]:
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self, busSystem):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
        self.busSystem = busSystem
        
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [123]:
class Queue:
    '''
    This class tracks the number of people in the queue 
    and the number of people being served. It also schedules
    when a customer starts being served and stops being served.
    '''
    # making a Queue with default 0 people waiting in a queue and 0 served
    def __init__(self, count_queue, count_served):
        self.count_queue = 0
        self.count_served = 0
    

In [124]:
class BusSystem:
    '''
     This class contains the queues (in this case only one 
     but you’ll need to make more than one for your project). 
     It also schedules the arrival of customers at the queue(s).
    '''
    
    def __init__(self, queue, service_time, arrival_distribution):
        
        self.queue = queue
        self.service_time = service_time
        self.arrival_distribution = arrival_distribution
        
        #for plotting the dats from a dictionary
        self.customers_per_time ={"time":[], "count_queue":[]}
        
    def schedule_arrival(self, schedule):
        # calculating the next arrival time
        next_arrival = schedule.now + self.arrival_distribution.rvs()
        # making an event of the arrival
        customer_arrives = Event(next_arrival, customer_serving)
        # add an event
        schedule.add_event_at(customer_arrives.timestamp, customer_arrives.function)
        # increase the queue
        self.queue.count_queue += 1
        schedule.now = customer_arrives.timestamp

In [125]:
# a function is neeeded for the Event() to accommodate 
# for the customers being served at the time of other customers arriving

def customer_serving(schedule):
    
    serve_rate = schedule.busSystem.service_time
    #for scheduling new arrival of passengers while others are being served
    schedule.now_ = schedule.now
    while schedule.now < schedule.now_ + serve_rate:
        schedule.busSystem.schedule_arrival(schedule)
    #update the queue count
    
    #adding data to the dictionary
    schedule.busSystem.queue.count_queue -= 1
    schedule.busSystem.customers_per_time["time"].append(schedule.now)
    schedule.busSystem.customers_per_time["count_queue"].append(schedule.busSystem.queue.count_queue)

## M/D/1 queue simulation

In [126]:
# setting parameters
arrival_rate = 1
service_rate = 5
run_until = 20

def run_simulation(arrival_rate, service_rate, run_until):
    # sample waiting times - exponential
    arrival_dist = sts.expon(scale=1/arrival_rate)
    # making a queue
    queue = Queue(0,0)
    # making a bus system object
    busSystem = BusSystem(queue, arrival_dist, service_rate)
    # setting up a schedule, bus system object
    schedule = Schedule(busSystem)
    busSystem.schedule_arrival(schedule)
    
    #running the simulation until set time, running next event
    while schedule.next_arrival() <= run_until:
        schedule.run_next_event()
    
    #plotting the data
    plt.plot(busSystem.customers_per_time["time"], busSystem.customers_per_time["count_queue"] )
    plt.legend()
        
run_simulation(arrival_rate, service_rate, run_until)

AttributeError: 'int' object has no attribute 'rvs'

## Reflection

As seen from above, my code still has a bug I was not able to problem solve, however I tried to implement the best practices of coding by naming variables and objects appropriately, adding the necessary elements to implement the simulation. I have created 2 class objects and 2 functions to (one of them the simulation function itself). As per suggestion of Nazar Yaremko (M22) I implemented the customer_serving function to accommodate for customers being served at the time while other customers are arriving.

## A simple test of the schedule

You do not need this code for your pre-class work. It demonstrates that the schedule can add and run events.

In [120]:
def print_and_add(schedule, string):
    print(f'At time {schedule.now}:', string)
    schedule.add_event_after(0.25, print_and_add, 'Another event')
            
schedule = Schedule()
schedule.print_events()

schedule.add_event_at(0.5, print_and_add, 'First event')
schedule.add_event_at(1.2, print_and_add, 'Second event')
schedule.print_events()

schedule.run_next_event()
schedule.print_events()

schedule.run_next_event()
schedule.print_events()

schedule.run_next_event()
schedule.print_events()

TypeError: __init__() missing 1 required positional argument: 'busSystem'

# The cuckoo clock simulation

You do not need this code for your pre-class work but you can use it as a starting point for implementing your queue simulation.

In [75]:
class Cuckoo:
    def __init__(self, interval):
        self.interval = interval
        self.hour = 0
    
    def make_some_noise(self, schedule):
        self.hour += 1
        print('Cuckoo! ' * self.hour)
        if self.hour == 12:
            self.hour = 0

class Pendulum:
    def __init__(self, cuckoo):
        self.cuckoo = cuckoo
    
    def tick(self, schedule, counter):
        counter += 1
        print(f'tick {counter} at time = {schedule.now}')
        schedule.add_event_after(0.5, self.tock, counter)
        
    def tock(self, schedule, counter):
        print(f'tock {counter} at time = {schedule.now}')
        schedule.add_event_after(0.5, self.tick, counter)
        if counter % self.cuckoo.interval == 0:
            schedule.add_event_after(0, self.cuckoo.make_some_noise)

class Clock:
    def __init__(self):
        self.cuckoo = Cuckoo(interval=5)
        self.pendulum = Pendulum(self.cuckoo)
    
    def run(self, schedule):
        schedule.add_event_at(0.5, self.pendulum.tick, counter=0)

def run_simulation(run_until):
    schedule = Schedule()
    clock = Clock()
    clock.run(schedule)
    while schedule.next_event_time() <= run_until:
        schedule.run_next_event()

In [4]:
run_simulation(run_until=10)

tick 1 at time = 0.5
tock 1 at time = 1.0
tick 2 at time = 1.5
tock 2 at time = 2.0
tick 3 at time = 2.5
tock 3 at time = 3.0
tick 4 at time = 3.5
tock 4 at time = 4.0
tick 5 at time = 4.5
tock 5 at time = 5.0
Cuckoo! 
tick 6 at time = 5.5
tock 6 at time = 6.0
tick 7 at time = 6.5
tock 7 at time = 7.0
tick 8 at time = 7.5
tock 8 at time = 8.0
tick 9 at time = 8.5
tock 9 at time = 9.0
tick 10 at time = 9.5
tock 10 at time = 10.0
Cuckoo! Cuckoo! 
